# Import the libraries

In [ ]:
from rdflib import Graph, Literal, term, URIRef, RDF
from rdflib.namespace import Namespace, NamespaceManager
import pyshacl
import re
import os
from rdflib.namespace import Namespace, NamespaceManager
from NLP2SHACL import write_shacl

# Load the graph

In [ ]:
# Load the data graph
data_graph = Graph()
data_graph.parse("InstanceModel/brick_model.ttl", format="turtle")

# Validation Function

In [ ]:
def validate_graph(data_graph, shape_graph):
    shapes_file_format = 'turtle'
    data_file_format = 'turtle'

    res = pyshacl.validate(data_graph, shacl_graph = shape_graph,
                                     data_graph_format = data_file_format,
                                     shacl_graph_format = shapes_file_format,
                                     inference='both')

    return res


# BOT GUID Example
Requirement:
Each bot:Space must have one and only guid and that guid must be a string. 

In [ ]:
# Load the Shape File
shape_graph = Graph()
shape_graph.parse("Shape/ifc_guid.ttl", format="turtle")

In [ ]:
res = validate_graph(data_graph, shape_graph)
res[0]

### Load the invalid data graph

In [ ]:
data_graph_invalid = Graph()
data_graph_invalid.parse("InstanceModel/brick_model_invalid.ttl", format="turtle")

In [ ]:
res = validate_graph(data_graph_invalid, shape_graph)
res[0]

# Brick VAV Example (point count)
Each brick:VAV must have two Points.

## Using NLP2SHACL tool

In [ ]:
# Generate the SHACL
requirement_txt= """
   R1 : Every HVAC_Zone should hasPoint at least 2.
   """

shacl_txt = write_shacl(requirement_txt)

print('SHACL text generated:\n {0}'.format(shacl_txt))

## Run the SHACL against the data graph

In [ ]:
# Load the generated shacl text and run validation
shape_graph = shacl_txt
res = validate_graph(data_graph, shape_graph)

print(res[2])

# Brick VAV Point Type Example

Each brick:VAV must have a brick:Zone_Air_Temperature_Sensor and a brick:Zone_Air_Temperature_Setpoint.

In [ ]:
# Generate the SHACL
requirement_txt= """
   R2 : Every VAV should hasPoint at least 1 Zone_Air_Temperature_Sensor and hasPoint at least 1 Zone_Air_Temperature_Setpoint.
   """

shacl_txt = write_shacl(requirement_txt)

print('SHACL text generated:\n {0}'.format(shacl_txt))

In [ ]:
# Load the generated shacl text and run validation
shape_graph = shacl_txt
res = validate_graph(data_graph, shape_graph)

print(res[2])


# Brick HVAC_Zone Example

Each HVAC_Zone should by fed by at least 1 VAV

In [ ]:
# Generate the SHACL
requirement_txt= """
   R3 : Every HVAC_Zone should feeds by at least 1 VAV.
   """

shacl_txt = write_shacl(requirement_txt)

print('SHACL text generated:\n {0}'.format(shacl_txt))

In [ ]:
# Load the generated shacl text and run validation
shape_graph = shacl_txt
res = validate_graph(data_graph, shape_graph)

print(res[2])